In [16]:
import torch
import torch.nn as nn

class BetterGRU(nn.Module):
    def __init__(self, input_size=6, hidden_size=128, num_layers=2, output_len=5, dropout=0.3):
        super().__init__()
        self.gru = nn.GRU(input_size, hidden_size, num_layers, batch_first=True, dropout=dropout)
        self.norm = nn.LayerNorm(hidden_size)
        self.fc = nn.Sequential(
            nn.Linear(hidden_size, hidden_size),
            nn.GELU(),
            nn.Linear(hidden_size, output_len * 2)
        )

    def forward(self, x):
        out, _ = self.gru(x)                   # out: [B, T, H]
        last_hidden = out[:, -1]               # Take last time step output
        last_hidden = self.norm(last_hidden)
        pred = self.fc(last_hidden)
        return pred.view(-1, 5, 2)


In [13]:
import pickle
import numpy as np
from tqdm import tqdm

# --- Load original trajectory data ---
with open("car_trajectories.pkl", "rb") as f:
    raw_data = pickle.load(f)

processed_data = {}

# --- Compute [x, y, vx, vy, ax, ay] for each timestep ---
def kalman_like_features(points):
    coords = np.array([[x, y] for _, x, y in points])
    features = []
    for i in range(len(coords)):
        x, y = coords[i]
        if i == 0:
            vx, vy = 0, 0
            ax, ay = 0, 0
        elif i == 1:
            vx, vy = coords[i] - coords[i - 1]
            ax, ay = 0, 0
        else:
            vx, vy = coords[i] - coords[i - 1]
            vx_prev, vy_prev = coords[i - 1] - coords[i - 2]
            ax, ay = vx - vx_prev, vy - vy_prev
        features.append([x, y, vx, vy, ax, ay])
    return features

# --- Process and store only valid-length sequences ---
for tid, pts in tqdm(raw_data.items(), desc="🔁 Processing trajectories"):
    if len(pts) >= 15:
        processed_data[tid] = kalman_like_features(pts)

# --- Save enhanced trajectory file ---
with open("car_trajectories_kalman.pkl", "wb") as f:
    pickle.dump(processed_data, f)

print("✅ Saved enhanced hybrid Kalman-GRU trajectory data to car_trajectories_kalman.pkl")


🔁 Processing trajectories: 100%|██████████| 7768/7768 [00:02<00:00, 2671.82it/s]


✅ Saved enhanced hybrid Kalman-GRU trajectory data to car_trajectories_kalman.pkl


In [15]:
import pickle
import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import TensorDataset, DataLoader
from sklearn.model_selection import train_test_split
from tqdm import tqdm

# --- Load and process enhanced trajectory data ---
with open("car_trajectories_kalman.pkl", "rb") as f:
    data = pickle.load(f)

# --- Parameters ---
input_len = 10
pred_len = 5

X_seqs = []
Y_seqs = []

for track_id, points in data.items():
    pts = np.array(points)  # shape: (T, 6)
    if len(pts) < input_len + pred_len:
        continue
    for i in range(len(pts) - input_len - pred_len + 1):
        X_seqs.append(pts[i:i + input_len])
        Y_seqs.append(pts[i + input_len:i + input_len + pred_len, :2])  # Only x, y in output

X = np.array(X_seqs)  # (N, 10, 6)
Y = np.array(Y_seqs)  # (N, 5, 2)
print(f"Prepared dataset: X={X.shape}, Y={Y.shape}")

# --- Train-val split ---
X_train, X_val, Y_train, Y_val = train_test_split(X, Y, test_size=0.2, random_state=42)

# --- Torch datasets ---
BATCH_SIZE = 64
train_ds = TensorDataset(torch.Tensor(X_train), torch.Tensor(Y_train))
val_ds = TensorDataset(torch.Tensor(X_val), torch.Tensor(Y_val))
train_dl = DataLoader(train_ds, batch_size=BATCH_SIZE, shuffle=True)
val_dl = DataLoader(val_ds, batch_size=BATCH_SIZE)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# --- GRU Model with 6D input ---
class HybridGRU(nn.Module):
    def __init__(self, input_size=6, hidden_size=128, num_layers=2, output_len=5, dropout=0.3):
        super().__init__()
        self.gru = nn.GRU(input_size, hidden_size, num_layers, batch_first=True, dropout=dropout)
        self.fc = nn.Linear(hidden_size, output_len * 2)

    def forward(self, x):
        _, hn = self.gru(x)
        out = self.fc(hn[-1])
        return out.view(-1, 5, 2)

# --- Custom velocity loss ---
def velocity_loss(pred, target):
    pred_vel = pred[:, 1:] - pred[:, :-1]
    target_vel = target[:, 1:] - target[:, :-1]
    return nn.functional.mse_loss(pred_vel, target_vel)

# --- Metric functions ---
def compute_metrics(pred, target):
    ade = ((pred - target) ** 2).sum(dim=2).sqrt().mean(dim=1).mean().item()
    fde = ((pred[:, -1] - target[:, -1]) ** 2).sum(dim=1).sqrt().mean().item()
    mae = nn.functional.l1_loss(pred, target).item()
    return ade, fde, mae

# --- Instantiate model ---
model = HybridGRU().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)
mse_loss = nn.MSELoss()
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, factor=0.5, patience=3)

# --- Training loop ---
best_val_loss = float('inf')
patience = 5
epochs_without_improvement = 0
checkpoint_path = 'best_hybrid_gru_model.pt'
EPOCHS = 100

for epoch in range(EPOCHS):
    print(f"\n--- Starting Epoch {epoch + 1}/{EPOCHS} ---")
    model.train()
    train_loss = 0
    for xb, yb in tqdm(train_dl, desc=f"\U0001F4AA Training Epoch {epoch+1}"):
        xb, yb = xb.to(device), yb.to(device)
        optimizer.zero_grad()
        pred = model(xb)
        loss = mse_loss(pred, yb) + 0.1 * velocity_loss(pred, yb)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
        optimizer.step()
        train_loss += loss.item() * xb.size(0)

    model.eval()
    val_loss, ade, fde, mae = 0, 0, 0, 0
    with torch.no_grad():
        for xb, yb in tqdm(val_dl, desc=f"\U0001F9EA Validating Epoch {epoch+1}"):
            xb, yb = xb.to(device), yb.to(device)
            pred = model(xb)
            loss = mse_loss(pred, yb) + 0.1 * velocity_loss(pred, yb)
            val_loss += loss.item() * xb.size(0)
            batch_ade, batch_fde, batch_mae = compute_metrics(pred, yb)
            ade += batch_ade * xb.size(0)
            fde += batch_fde * xb.size(0)
            mae += batch_mae * xb.size(0)

    train_loss /= len(train_dl.dataset)
    val_loss /= len(val_dl.dataset)
    ade /= len(val_dl.dataset)
    fde /= len(val_dl.dataset)
    mae /= len(val_dl.dataset)
    scheduler.step(val_loss)

    print(f"Epoch {epoch + 1} - Train Loss: {train_loss:.4f}, Val Loss: {val_loss:.4f} | "
          f"ADE: {ade:.4f}, FDE: {fde:.4f}, MAE: {mae:.4f}")

    # --- Early stopping ---
    if val_loss < best_val_loss:
        best_val_loss = val_loss
        epochs_without_improvement = 0
        torch.save(model.state_dict(), checkpoint_path)
        print("✔️  New best model saved.")
    else:
        epochs_without_improvement += 1
        print(f"⚠️  No improvement. {epochs_without_improvement} epochs without improvement.")

    if epochs_without_improvement >= patience:
        print("⏹️ Early stopping triggered.")
        break


Prepared dataset: X=(609284, 10, 6), Y=(609284, 5, 2)
Using device: cuda

--- Starting Epoch 1/100 ---


🧪 Validating Epoch 1: 100%|██████████| 1905/1905 [00:01<00:00, 1364.60it/s]


Epoch 1 - Train Loss: 0.1423, Val Loss: 0.0104 | ADE: 0.0730, FDE: 0.0943, MAE: 0.0443
✔️  New best model saved.

--- Starting Epoch 2/100 ---


🧪 Validating Epoch 2: 100%|██████████| 1905/1905 [00:01<00:00, 1428.16it/s]


Epoch 2 - Train Loss: 0.0134, Val Loss: 0.0115 | ADE: 0.0905, FDE: 0.1109, MAE: 0.0543
⚠️  No improvement. 1 epochs without improvement.

--- Starting Epoch 3/100 ---


...

Epoch 18 - Train Loss: 0.0078, Val Loss: 0.0078 | ADE: 0.0619, FDE: 0.0768, MAE: 0.0360
⚠️  No improvement. 3 epochs without improvement.

--- Starting Epoch 19/100 ---


🧪 Validating Epoch 19: 100%|██████████| 1905/1905 [00:01<00:00, 1199.31it/s]


Epoch 19 - Train Loss: 0.0078, Val Loss: 0.0076 | ADE: 0.0529, FDE: 0.0688, MAE: 0.0314
⚠️  No improvement. 4 epochs without improvement.

--- Starting Epoch 20/100 ---


🧪 Validating Epoch 20: 100%|██████████| 1905/1905 [00:01<00:00, 1414.44it/s]

Epoch 20 - Train Loss: 0.0071, Val Loss: 0.0074 | ADE: 0.0570, FDE: 0.0837, MAE: 0.0352
⚠️  No improvement. 5 epochs without improvement.
⏹️ Early stopping triggered.


In [17]:
from tqdm import tqdm
import torch.nn.functional as F

def velocity_loss(pred, target):
    pred_vel = pred[:, 1:] - pred[:, :-1]
    target_vel = target[:, 1:] - target[:, :-1]
    return F.mse_loss(pred_vel, target_vel)

def compute_metrics(pred, target):
    ade = torch.mean(torch.norm(pred - target, dim=2))
    fde = torch.mean(torch.norm(pred[:, -1] - target[:, -1], dim=1))
    mae = F.l1_loss(pred, target)
    return ade.item(), fde.item(), mae.item()


In [19]:
model = BetterGRU().to(device)
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-3, weight_decay=1e-4)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, factor=0.5, patience=3)
scaler = torch.cuda.amp.GradScaler()

EPOCHS = 100
patience = 7
best_val_loss = float("inf")
epochs_without_improvement = 0
checkpoint_path = "best_gru_hybrid.pt"

mse_loss = nn.MSELoss()

for epoch in range(EPOCHS):
    print(f"\n--- Epoch {epoch+1}/{EPOCHS} ---")
    model.train()
    train_loss = 0.0

    for xb, yb in tqdm(train_dl, desc="Training", leave=False):
        xb, yb = xb.to(device), yb.to(device)
        optimizer.zero_grad()
        with torch.cuda.amp.autocast():
            pred = model(xb)
            loss = mse_loss(pred, yb) + 0.1 * velocity_loss(pred, yb)
        scaler.scale(loss).backward()
        scaler.unscale_(optimizer)
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
        scaler.step(optimizer)
        scaler.update()
        train_loss += loss.item() * xb.size(0)

    model.eval()
    val_loss, ade, fde, mae = 0, 0, 0, 0
    with torch.no_grad():
        for xb, yb in tqdm(val_dl, desc="Validation", leave=False):
            xb, yb = xb.to(device), yb.to(device)
            pred = model(xb)
            loss = mse_loss(pred, yb) + 0.1 * velocity_loss(pred, yb)
            val_loss += loss.item() * xb.size(0)
            batch_ade, batch_fde, batch_mae = compute_metrics(pred, yb)
            ade += batch_ade * xb.size(0)
            fde += batch_fde * xb.size(0)
            mae += batch_mae * xb.size(0)

    train_loss /= len(train_dl.dataset)
    val_loss /= len(val_dl.dataset)
    ade /= len(val_dl.dataset)
    fde /= len(val_dl.dataset)
    mae /= len(val_dl.dataset)

    scheduler.step(val_loss)
    print(f"Train Loss: {train_loss:.4f}, Val Loss: {val_loss:.4f} | ADE: {ade:.4f}, FDE: {fde:.4f}, MAE: {mae:.4f}")

    if val_loss < best_val_loss:
        best_val_loss = val_loss
        epochs_without_improvement = 0
        torch.save(model.state_dict(), checkpoint_path)
        print("✔️  New best model saved.")
    else:
        epochs_without_improvement += 1
        print(f"⚠️  No improvement. {epochs_without_improvement} epoch(s) without improvement.")

    if epochs_without_improvement >= patience:
        print("⏹️  Early stopping triggered.")
        break


/tmp/ipykernel_513928/2964829529.py:4: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = torch.cuda.amp.GradScaler()



--- Epoch 1/100 ---


Training:   0%|          | 0/7617 [00:00<?, ?it/s]/tmp/ipykernel_513928/2964829529.py:22: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast():


Train Loss: 0.1272, Val Loss: 0.0106 | ADE: 0.0749, FDE: 0.0943, MAE: 0.0464
✔️  New best model saved.

--- Epoch 2/100 ---


Train Loss: 0.0165, Val Loss: 0.0139 | ADE: 0.1088, FDE: 0.1302, MAE: 0.0671
⚠️  No improvement. 1 epoch(s) without improvement.

--- Epoch 3/100 ---


...

Train Loss: 0.0070, Val Loss: 0.0079 | ADE: 0.0571, FDE: 0.0709, MAE: 0.0343
⚠️  No improvement. 3 epoch(s) without improvement.

--- Epoch 21/100 ---


Train Loss: 0.0068, Val Loss: 0.0076 | ADE: 0.0527, FDE: 0.0673, MAE: 0.0302
⚠️  No improvement. 4 epoch(s) without improvement.

--- Epoch 22/100 ---


Train Loss: 0.0063, Val Loss: 0.0072 | ADE: 0.0484, FDE: 0.0673, MAE: 0.0282
⚠️  No improvement. 5 epoch(s) without improvement.

--- Epoch 23/100 ---


Train Loss: 0.0061, Val Loss: 0.0074 | ADE: 0.0501, FDE: 0.0652, MAE: 0.0289
⚠️  No improvement. 6 epoch(s) without improvement.

--- Epoch 24/100 ---


Train Loss: 0.0060, Val Loss: 0.0073 | ADE: 0.0500, FDE: 0.0678, MAE: 0.0293
⚠️  No improvement. 7 epoch(s) without improvement.
⏹️  Early stopping triggered.


In [20]:
import torch
import torch.nn as nn
from torch.utils.data import TensorDataset, DataLoader
from sklearn.model_selection import train_test_split
from tqdm import tqdm
import numpy as np
import pickle

# --- Load Data ---
with open("car_trajectories.pkl", "rb") as f:
    data = pickle.load(f)

input_len = 10
pred_len = 5

X_seqs, Y_seqs = [], []
for points in data.values():
    coords = np.array([[p[1], p[2]] for p in points])
    if len(coords) < input_len + pred_len:
        continue
    vx = np.diff(coords[:, 0], prepend=coords[0, 0])
    vy = np.diff(coords[:, 1], prepend=coords[0, 1])
    ax = np.diff(vx, prepend=vx[0])
    ay = np.diff(vy, prepend=vy[0])
    features = np.stack([coords[:, 0], coords[:, 1], vx, vy, ax, ay], axis=-1)
    for i in range(len(features) - input_len - pred_len + 1):
        X_seqs.append(features[i:i + input_len])
        Y_seqs.append(coords[i + input_len:i + input_len + pred_len])

X = np.array(X_seqs)
Y = np.array(Y_seqs)
X_train, X_val, Y_train, Y_val = train_test_split(X, Y, test_size=0.2, random_state=42)

# --- DataLoader ---
BATCH_SIZE = 128
dev = torch.device("cuda" if torch.cuda.is_available() else "cpu")
train_ds = TensorDataset(torch.Tensor(X_train), torch.Tensor(Y_train))
val_ds = TensorDataset(torch.Tensor(X_val), torch.Tensor(Y_val))
train_dl = DataLoader(train_ds, batch_size=BATCH_SIZE, shuffle=True)
val_dl = DataLoader(val_ds, batch_size=BATCH_SIZE)

# --- Model ---
class BetterLSTM(nn.Module):
    def __init__(self, input_size=6, hidden_size=128, num_layers=2, output_len=5, dropout=0.3):
        super().__init__()
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True, dropout=dropout)
        self.norm = nn.LayerNorm(hidden_size)
        self.fc = nn.Sequential(
            nn.Linear(hidden_size, hidden_size),
            nn.GELU(),
            nn.Linear(hidden_size, output_len * 2)
        )

    def forward(self, x):
        out, _ = self.lstm(x)
        last_hidden = out[:, -1]
        last_hidden = self.norm(last_hidden)
        pred = self.fc(last_hidden)
        return pred.view(-1, 5, 2)

model = BetterLSTM().to(dev)
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, factor=0.5, patience=2)
mse_loss = nn.MSELoss()

def velocity_loss(pred, target):
    pred_vel = pred[:, 1:] - pred[:, :-1]
    target_vel = target[:, 1:] - target[:, :-1]
    return nn.functional.mse_loss(pred_vel, target_vel)

def compute_metrics(pred, target):
    ade = ((pred - target) ** 2).sum(dim=2).sqrt().mean(dim=1).mean().item()
    fde = ((pred[:, -1] - target[:, -1]) ** 2).sum(dim=1).sqrt().mean().item()
    mae = (pred - target).abs().mean().item()
    return ade, fde, mae

# --- Training Loop ---
best_val_loss = float('inf')
patience = 5
epochs_without_improvement = 0
EPOCHS = 100
checkpoint_path = 'best_lstm_model.pt'

for epoch in range(EPOCHS):
    print(f"\n🔁 Epoch {epoch + 1}/{EPOCHS}")

    model.train()
    train_loss = 0
    for xb, yb in tqdm(train_dl, desc="Training"):
        xb, yb = xb.to(dev), yb.to(dev)
        optimizer.zero_grad()
        pred = model(xb)
        loss = mse_loss(pred, yb) + 0.1 * velocity_loss(pred, yb)
        loss.backward()
        nn.utils.clip_grad_norm_(model.parameters(), 1.0)
        optimizer.step()
        train_loss += loss.item() * xb.size(0)

    model.eval()
    val_loss, ade, fde, mae = 0, 0, 0, 0
    with torch.no_grad():
        for xb, yb in tqdm(val_dl, desc="Validation"):
            xb, yb = xb.to(dev), yb.to(dev)
            pred = model(xb)
            loss = mse_loss(pred, yb) + 0.1 * velocity_loss(pred, yb)
            val_loss += loss.item() * xb.size(0)
            b_ade, b_fde, b_mae = compute_metrics(pred, yb)
            ade += b_ade * xb.size(0)
            fde += b_fde * xb.size(0)
            mae += b_mae * xb.size(0)

    train_loss /= len(train_dl.dataset)
    val_loss /= len(val_dl.dataset)
    ade /= len(val_dl.dataset)
    fde /= len(val_dl.dataset)
    mae /= len(val_dl.dataset)
    scheduler.step(val_loss)

    print(f"📊 Loss: Train={train_loss:.4f}, Val={val_loss:.4f} | ADE={ade:.3f}, FDE={fde:.3f}, MAE={mae:.3f}")

    if val_loss < best_val_loss:
        best_val_loss = val_loss
        epochs_without_improvement = 0
        torch.save(model.state_dict(), checkpoint_path)
        print("✅ Saved best model")
    else:
        epochs_without_improvement += 1
        print(f"⚠️ No improvement ({epochs_without_improvement}/{patience})")

    if epochs_without_improvement >= patience:
        print("⏹️ Early stopping triggered.")
        break


🔁 Epoch 1/100


Validation: 100%|██████████| 953/953 [00:01<00:00, 906.03it/s]


📊 Loss: Train=0.2064, Val=0.0236 | ADE=0.172, FDE=0.187, MAE=0.112
✅ Saved best model

🔁 Epoch 2/100


Validation: 100%|██████████| 953/953 [00:01<00:00, 931.00it/s]


📊 Loss: Train=0.0174, Val=0.0132 | ADE=0.106, FDE=0.124, MAE=0.067
✅ Saved best model

🔁 Epoch 3/100


...

📊 Loss: Train=0.0045, Val=0.0059 | ADE=0.041, FDE=0.062, MAE=0.024
⚠️ No improvement (3/5)

🔁 Epoch 37/100


Validation: 100%|██████████| 953/953 [00:01<00:00, 926.68it/s]


📊 Loss: Train=0.0045, Val=0.0059 | ADE=0.042, FDE=0.062, MAE=0.025
⚠️ No improvement (4/5)

🔁 Epoch 38/100


Validation: 100%|██████████| 953/953 [00:01<00:00, 926.58it/s]

📊 Loss: Train=0.0045, Val=0.0059 | ADE=0.042, FDE=0.062, MAE=0.024
⚠️ No improvement (5/5)
⏹️ Early stopping triggered.
